In [52]:
# !pip install openai pandas python-dotenv

import pandas as pd
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=OPENAI_API_KEY)

print("OpenAI API Key Loaded:", OPENAI_API_KEY is not None)


OpenAI API Key Loaded: True


In [53]:
data = [
    {"trend_name": "bucket hat", "predicted_trend_score": 0.039, "forecasted_trend_score": 0.110,
     "trendDirection": "stable"},
    {"trend_name": "crop top", "predicted_trend_score": 0.400, "forecasted_trend_score": 0.405,
     "trendDirection": "stable"},
    {"trend_name": "denim overalls", "predicted_trend_score": 0.516, "forecasted_trend_score": 0.511,
     "trendDirection": "stable"},
    {"trend_name": "leather boots", "predicted_trend_score": 0.218, "forecasted_trend_score": 0.210,
     "trendDirection": "stable"}
]
df_trends = pd.DataFrame(data)
df_trends


,trend_name,predicted_trend_score,forecasted_trend_score,trendDirection
0,bucket hat,0.039,0.110,stable
1,crop top,0.400,0.405,stable
2,denim overalls,0.516,0.511,stable
3,leather boots,0.218,0.210,stable


In [56]:
def call_detail_agent(df):
    records = df[['trend_name','predicted_trend_score','forecasted_trend_score','trendDirection']].to_dict(orient='records')
    prompt = (
        "You are a Responsible AI Auditor. Analyze the following fashion trend data and "
        "identify potential issues regarding fairness, bias, transparency, explainability, "
        "forecast reliability, or data quality. Output as concise bullet points separated by semicolons.\n\n"
        f"Trend Data:\n{records}\n\nOutput points separated by semicolons."
    )

    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role":"user","content":prompt}],
        temperature=0.3,
        max_tokens=400
    )

    # Extract just the text content
    detail_points = resp.choices[0].message.content.strip()
    return detail_points  # <-- return only the text


In [57]:
detail_text = call_detail_agent(df_trends)
print(detail_text)

- Low predicted trend scores for some items (e.g., bucket hat) may indicate a lack of interest or visibility, potentially leading to unfair representation in trend forecasting; 
- The small range between predicted and forecasted scores for most trends suggests a lack of variability, which may limit the model's ability to capture emerging styles or shifts in consumer preferences; 
- The model's reliance on historical data may introduce bias, favoring established trends over innovative or niche styles; 
- Lack of demographic breakdown in trend data raises concerns about transparency and fairness, as certain groups may be underrepresented in trend forecasting; 
- The stability in trend direction across all items may indicate a conservative approach to forecasting, potentially overlooking disruptive trends; 
- The absence of explainability regarding how scores were derived limits stakeholders' ability to understand and trust the model's predictions; 
- Data quality issues may arise if the 

In [60]:
def call_summary_agent(detail_text):
    """
    Summarize detailed audit points into exactly 10 concise points.
    detail_text: str output from Detail Agent
    """
    prompt = (
        "You are a Responsible AI Auditor. Summarize the following audit points into, mention specific items as well"
        "a concise report containing exactly 10 distinct points, semicolon-separated:\n\n"
        f"{detail_text}"
    )

    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
        max_tokens=300
    )

    # Extract only the text content
    try:
        summary_text = resp.choices[0].message.content.strip()
    except:
        summary_text = str(resp.choices[0])

    print("=== Summary Agent Output ===")
    print(summary_text)
    return summary_text





In [61]:
summary_points = call_summary_agent(detail_text)

=== Summary Agent Output ===
1. Low predicted trend scores for items like bucket hats indicate potential lack of interest or visibility;  
2. Small range between predicted and forecasted scores suggests limited variability in capturing emerging styles;  
3. Model reliance on historical data may introduce bias, favoring established trends over innovative styles;  
4. Absence of demographic breakdown in trend data raises concerns about transparency and fairness;  
5. Stability in trend direction across items indicates a conservative forecasting approach, risking oversight of disruptive trends;  
6. Lack of explainability in score derivation limits stakeholder understanding and trust in predictions;  
7. Data quality issues may arise from outdated or non-diverse input data, affecting forecast reliability;  
8. Potential overfitting to past trends risks inaccurate predictions in a rapidly changing fashion landscape;  
9. Unfair representation in trend forecasting may result from low visibi

In [66]:
def audit_trends_text(df):
    # Step 1: Detail Agent
    detail_text = call_detail_agent(df)

    # Step 2: Summary Agent
    summary_text = call_summary_agent(detail_text)

    print("\n=== Detail Agent Output ===")
    print(detail_text)

    print("\n=== Summary Agent Output ===")
    print(summary_text)

    # Return both texts as tuple if needed
    return detail_text, summary_text





In [67]:
detail_text, summary_text = audit_trends_text(df_trends)

=== Summary Agent Output ===
1. Low predicted trend scores for 'bucket hat' indicate potential marginalization of this style;  
2. Small variance between predicted and forecasted trend scores for 'crop top' and 'denim overalls' suggests a reliable model;  
3. Lack of diversity in trend forecasting may be indicated by the small variance in scores;  
4. Stable trend direction across all items may overlook emerging trends or shifts in consumer preferences;  
5. Potential bias in trend analysis arises from the stable trend direction;  
6. Lack of transparency in methodology hinders trust in trend predictions;  
7. Absence of demographic data raises concerns about fairness and inclusivity;  
8. Trends may not reflect diverse consumer preferences due to lack of demographic data;  
9. Over-reliance on historical data could affect forecast reliability amid changing consumer behavior;  
10. Uncertain overall data quality impacts prediction accuracy without clear sources and methods.

=== Detail